In [3]:
import os
import cv2
import torch
from torch.utils.data import Dataset
import numpy as np

In [4]:
class PatchDataset(Dataset):
    def __init__(self, original_dir, filtered_dir, patch_size=3):
        """
        original_dir: Orijinal resimlerin bulunduğu klasör (ör. 'imgs')
        filtered_dir: Filtrelenmiş resimlerin bulunduğu klasör (ör. 'filtered_imgs')
        patch_size: Kullanılacak patch boyutu (3x3 için 3)
        """
        self.original_dir = original_dir
        self.filtered_dir = filtered_dir
        self.patch_size = patch_size
        
        # Dosyaları isim sırasına göre alıyoruz (jpeg/jpg ve png uzantıları için)
        self.original_files = sorted([os.path.join(original_dir, f) for f in os.listdir(original_dir)
                                        if f.lower().endswith(('.jpg', '.jpeg'))])
        self.filtered_files = sorted([os.path.join(filtered_dir, f) for f in os.listdir(filtered_dir)
                                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        
        assert len(self.original_files) == len(self.filtered_files), "Orijinal ve filtrelenmiş resim sayıları uyuşmuyor."
        
        # Her resim için patch çıkarılabilecek merkezi piksel konumlarını hesaplıyoruz
        self.patch_indices = []
        for i, orig_file in enumerate(self.original_files):
            img = cv2.imread(orig_file, cv2.IMREAD_GRAYSCALE)
            h, w = img.shape
            # Kenar pikselleri hariç bırakılıyor (3x3 patch için)
            for r in range(1, h - 1):
                for c in range(1, w - 1):
                    self.patch_indices.append((i, r, c))
    
    def __len__(self):
        return len(self.patch_indices)
    
    def __getitem__(self, idx):
        image_idx, r, c = self.patch_indices[idx]
        orig_path = self.original_files[image_idx]
        filt_path = self.filtered_files[image_idx]
        
        # Gri tonlamalı resimleri okuyun
        orig_img = cv2.imread(orig_path, cv2.IMREAD_GRAYSCALE)
        filt_img = cv2.imread(filt_path, cv2.IMREAD_GRAYSCALE)
        
        # Piksel değerlerini [-1, +1] aralığına normalize edin
        orig_img = orig_img.astype(np.float32) / 127.5 - 1.0
        filt_img = filt_img.astype(np.float32) / 127.5 - 1.0
        
        half = self.patch_size // 2
        # 3x3 patch'i orijinal resimden çıkarın (merkez (r, c))
        patch = orig_img[r - half: r + half + 1, c - half: c + half + 1]
        patch = patch.flatten()  # 9 elemanlı vektör
        
        # Hedef: filtrelenmiş resimdeki aynı konumdaki merkezi piksel
        target = filt_img[r, c]
        
        # Torch tensorlerine dönüştürün
        patch_tensor = torch.from_numpy(patch).float()
        target_tensor = torch.tensor(target).float()
        
        return patch_tensor, target_tensor

In [5]:
# Dataset'i oluşturun
dataset = PatchDataset('imgs', 'filtered_imgs', patch_size=3)
print("Dataset örnek sayısı:", len(dataset))
sample_patch, sample_target = dataset[0]
print("Bir patch'in boyutu:", sample_patch.shape, "Hedef piksel değeri:", sample_target.item())


Dataset örnek sayısı: 645160
Bir patch'in boyutu: torch.Size([9]) Hedef piksel değeri: 0.17647063732147217


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Kullanılan cihaz:", device)

In [ ]:
# Model tanımı: Feedforward ANN
class FFANN(nn.Module):
    def __init__(self):
        super(FFANN, self).__init__()
        self.fc1 = nn.Linear(9, 21)
        self.fc2 = nn.Linear(21, 17)
        self.fc3 = nn.Linear(17, 13)
        self.fc4 = nn.Linear(13, 9)
        self.fc5 = nn.Linear(9, 1)
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.tanh(self.fc1(x))
        x = self.tanh(self.fc2(x))
        x = self.tanh(self.fc3(x))
        x = self.tanh(self.fc4(x))
        x = self.tanh(self.fc5(x))
        return x

# Dataset'iniz zaten oluşturulduğuna göre DataLoader ile verileri gruplayın:
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

# Model, kayıp fonksiyonu ve optimizörü tanımlayın
model = FFANN().to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Eğitim döngüsü
num_epochs = 10  # İhtiyaç duyduğunuz kadar epoch ile deneyebilirsiniz
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(dataloader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()         # Gradients'i sıfırla
        outputs = model(inputs)         # İleri besleme
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()                 # Geri yayılım
        optimizer.step()                # Ağırlıkları güncelle
        
        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print(f"Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss/100:.4f}")
            running_loss = 0.0

print("Eğitim tamamlandı!")

Epoch 1, Batch 100, Loss: 0.1357
Epoch 1, Batch 200, Loss: 0.1059
Epoch 1, Batch 300, Loss: 0.1077
Epoch 1, Batch 400, Loss: 0.1071
Epoch 1, Batch 500, Loss: 0.1077
Epoch 1, Batch 600, Loss: 0.1076
Epoch 1, Batch 700, Loss: 0.1066
Epoch 1, Batch 800, Loss: 0.1059
Epoch 1, Batch 900, Loss: 0.1047
Epoch 1, Batch 1000, Loss: 0.1049
Epoch 1, Batch 1100, Loss: 0.1043
Epoch 1, Batch 1200, Loss: 0.1050
Epoch 1, Batch 1300, Loss: 0.1058
Epoch 1, Batch 1400, Loss: 0.1028
Epoch 1, Batch 1500, Loss: 0.1013
Epoch 1, Batch 1600, Loss: 0.1018
Epoch 1, Batch 1700, Loss: 0.1028
Epoch 1, Batch 1800, Loss: 0.1003
Epoch 1, Batch 1900, Loss: 0.0963
Epoch 1, Batch 2000, Loss: 0.0975
Epoch 1, Batch 2100, Loss: 0.0940
Epoch 1, Batch 2200, Loss: 0.0927
Epoch 1, Batch 2300, Loss: 0.0878
Epoch 1, Batch 2400, Loss: 0.0840
Epoch 1, Batch 2500, Loss: 0.0825
Epoch 2, Batch 100, Loss: 0.0784
Epoch 2, Batch 200, Loss: 0.0702


KeyboardInterrupt: 